## Purpose
The purpose of this notebook to load in all the images and their associated labels, then make a new image which is cropped based on the labels (essentially giving us our region of interest). Those images will be used to train the K-Means clustering, and also will be used for generating synthetic "unhealthy" images.

The following cell imports the needed libraries

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image

The following cell creates the directory where all the png images will be saved.

In [2]:
try:
    if not os.path.exists('cropped_png_images'):
        os.makedirs('cropped_png_images')
except OSError:
    print('Error creating data directory.')

The following cell takes an image and its associated label and crops it to the ROI, then resizes it.

In [3]:
import os
from PIL import Image

def crop_and_resize(image_path, label_path):
    """Crops a 256x256 image to the region of interest (ROI) specified by the label file and resizes it back to 256x256."""
    # Verify that the image and label file have matching base names.
    image_base = os.path.splitext(os.path.basename(image_path))[0]
    label_base = os.path.splitext(os.path.basename(label_path))[0]
    if image_base != label_base:
        raise ValueError("Image file and label file names do not match.")

    # Open the image.
    image = Image.open(image_path)
    if image.size != (256, 256):
        raise ValueError("The input image is not 256x256 in size.")

    # Read and parse the label file.
    with open(label_path, 'r') as f:
        line = f.readline().strip()
        parts = line.split()
        if len(parts) != 5:
            raise ValueError("Label file does not contain exactly 5 values.")
        # Parse values: ignoring the first value which represents the class label.
        try:
            _, x_center, y_center, box_width, box_height = parts
            x_center = float(x_center)
            y_center = float(y_center)
            box_width = float(box_width)
            box_height = float(box_height)
        except ValueError:
            raise ValueError("One or more of the coordinate values are not valid floats.")

    # Calculate pixel coordinates for cropping.
    img_width, img_height = image.size
    left   = int((x_center - box_width / 2) * img_width)
    top    = int((y_center - box_height / 2) * img_height)
    right  = int((x_center + box_width / 2) * img_width)
    bottom = int((y_center + box_height / 2) * img_height)
    
    # Ensure coordinates are within image boundaries.
    left = max(0, left)
    top = max(0, top)
    right = min(img_width, right)
    bottom = min(img_height, bottom)

    # Crop the image to the region of interest.
    cropped_image = image.crop((left, top, right, bottom))

    # Resize cropped image back to 256x256 using a high-quality filter.
    resized_image = cropped_image.resize((256, 256), Image.Resampling.LANCZOS)

    return resized_image

The following cell iterates through the images and their labels to create cropped ROIs.

In [4]:
def crop_all_images(image_input_dir, label_input_dir, output_dir): 
    """Crops all images to the region of interest (ROI) based on labels, resizes them, and saves them as PNG in output_dir."""
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for filename in os.listdir(image_input_dir):
        img_path = os.path.join(image_input_dir, filename)

        # Check if the image is a PNG file
        if filename.lower().endswith(".png"):
            label_filename = os.path.splitext(filename)[0] + ".txt"
            label_path = os.path.join(label_input_dir, label_filename)

            # Check if the associated label file exists
            if os.path.exists(label_path):
                try:
                    # Crop and resize the image
                    result_image = crop_and_resize(img_path, label_path)
                    
                    # Save the processed image in the output directory
                    result_image.save(os.path.join(output_dir, filename), format="PNG")
                except Exception as e:
                    print(f"Error processing {filename}: {e}")
            else:
                print(f"Skipping {filename}: Label file {label_filename} not found.")


Function call cell.

In [6]:
crop_all_images("/data/ai_club/team_13_2024-25/VIPR/Data/YOLO_images_total/", "/data/ai_club/team_13_2024-25/VIPR/Data/YOLO_labels_total/", "/data/ai_club/team_13_2024-25/VIPR/Data/ROI_cropped_images")

The following cell takes a cropped image, and makes synthetic "unhealthy" images of vocal cords, by making images where one side is stretched vertically by 20%.

In [7]:
import os
from PIL import Image

def split_and_stretch_images(input_dir, output_dir):
    """
    Processes all cropped/resized PNG images in input_dir by creating two variants:
    
    1. Left-Stretched Variant:
       - Splits the image vertically.
       - Stretches the left half vertically by 20% (i.e. new height = original height * 1.2).
       - Pastes the stretched left half with the original right half (vertically centered).
       
    2. Right-Stretched Variant:
       - Splits the image vertically.
       - Stretches the right half vertically by 20%.
       - Pastes the stretched right half with the original left half (vertically centered).
    
    In each case, the composite image is then resized back to the original dimensions (assumed 256x256)
    to ensure a consistent size for machine learning processing.
    
    The resulting images are saved as PNGs in output_dir with intuitive filenames.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for filename in os.listdir(input_dir):
        if filename.lower().endswith(".png"):
            img_path = os.path.join(input_dir, filename)
            try:
                img = Image.open(img_path)
                original_width, original_height = img.size  # Expected to be 256x256
                mid = original_width // 2  # Split point
                
                # Split the image into left and right halves.
                left_half = img.crop((0, 0, mid, original_height))
                right_half = img.crop((mid, 0, original_width, original_height))
                
                # ----- Left-Stretched Variant -----
                # Stretch left half vertically by 20%
                new_left_height = int(original_height * 1.2)
                left_stretched = left_half.resize((left_half.width, new_left_height), Image.Resampling.LANCZOS)
                # Determine composite height: use the taller height between stretched and original half.
                composite_height = max(new_left_height, original_height)
                # Create a new blank image for the composite.
                composite_left = Image.new(img.mode, (original_width, composite_height))
                # Paste the stretched left half at the top-left corner.
                composite_left.paste(left_stretched, (0, 0))
                # Vertically center the right half.
                right_top = (composite_height - original_height) // 2
                composite_left.paste(right_half, (mid, right_top))
                # Resize composite back to original size (e.g., 256x256).
                composite_left_resized = composite_left.resize((original_width, original_height), Image.Resampling.LANCZOS)
                # Save with an intuitive filename.
                left_output_name = os.path.splitext(filename)[0] + "_left_stretched.png"
                composite_left_resized.save(os.path.join(output_dir, left_output_name), format="PNG")
                
                # ----- Right-Stretched Variant -----
                # Stretch right half vertically by 20%
                new_right_height = int(original_height * 1.2)
                right_stretched = right_half.resize((right_half.width, new_right_height), Image.Resampling.LANCZOS)
                composite_height = max(new_right_height, original_height)
                composite_right = Image.new(img.mode, (original_width, composite_height))
                # Vertically center the left half.
                left_top = (composite_height - original_height) // 2
                composite_right.paste(left_half, (0, left_top))
                # Paste the stretched right half at the top-right (starting at the midpoint).
                composite_right.paste(right_stretched, (mid, 0))
                composite_right_resized = composite_right.resize((original_width, original_height), Image.Resampling.LANCZOS)
                right_output_name = os.path.splitext(filename)[0] + "_right_stretched.png"
                composite_right_resized.save(os.path.join(output_dir, right_output_name), format="PNG")
                
            except Exception as e:
                print(f"Skipping {filename}: {e}")

# Example usage:
# split_and_stretch_images("cropped_png_images", "stretched_png_images")


In [8]:
split_and_stretch_images("/data/ai_club/team_13_2024-25/VIPR/Data/ROI_cropped_images", "/data/ai_club/team_13_2024-25/VIPR/Data/synthetic_paralyzed_data")